In [72]:
from lightjob.db import DB, SUCCESS, RUNNING
import pandas as pd
from scipy.stats import skew
import os
import numpy as np
from collections import defaultdict

from bokeh.charts import Scatter, show, Histogram
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [41]:
filename = '../.lightjob/'

In [42]:
db = DB()
db.load(filename)

# Contraction and sparsity

In [103]:
stats_list = [
    u'intdim_mle',
    u'skew',
    u'clusdiversity',
    u'multiplecorrelation',
    u'var',
    u'mean'
]

stat_name = 'contractive_coef'
stat_get = lambda j:j['content']['contractive_coef']

In [105]:
where = 'jobset5'
stats = defaultdict(list)
for j in db.jobs_with(state=SUCCESS, type='generation'):
    content = j['content']
    s = content['model_summary']
    jref = db.get_job_by_summary(s)
    if jref['where'] != where:
        continue
    if "stats" not in j:
        continue
    stats[stat_name].append(stat_get(jref))    
    d = j['stats']
    for k in stats_list:
        stats[k].append(d[k])

In [106]:
fig = Scatter(x=stat_name, y='mean', data=stats)
show(fig)

In [107]:
fig = Scatter(x=stat_name, y='var', data=stats)
show(fig)

In [108]:
fig = Scatter(x=stat_name, y='skew', data=stats)
show(fig)

In [111]:
fig = Histogram(stats, stat_name)
show(fig)

In [113]:
fig = Scatter(x=stat_name, y='multiplecorrelation', data=stats)
show(fig)

In [114]:
fig = Scatter(x=stat_name, y='clusdiversity', data=stats)
show(fig)

In [115]:
fig = Scatter(x=stat_name, y='intdim_mle', data=stats)
show(fig)

# Walkback

In [116]:
stats = defaultdict(list)
where = 'jobset17'
for j in db.jobs_with(state=SUCCESS, type='generation'):
    content = j['content']
    s = content['model_summary']
    jref = db.get_job_by_summary(s)
    if jref['where'] != where:
        continue
    if "stats" not in j:
        continue
    d = j['stats']
    for k, v in d.items():
        stats[k].append(v)
    stats['walkback'].append(jref['content']['walkback'])
    stats['denoise'].append(jref['content']['denoise'])

In [117]:
fig = Scatter(x='denoise', y='intdim_mle', data=stats)
show(fig)